# Setup


---



In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import builtins
import math
from google.colab import drive

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_gcp import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Installing, importing, and initializing PySpark

---



In [ ]:
%%capture
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [ ]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()
spark

# Copy some wiki data

---



In [ ]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
/ [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [ ]:
!mkdir title_bins
!mkdir body_bins
!mkdir anchor_bins

# Processing wikipedia

---



In [ ]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

# Extract the **title**, **body** and **anchor text**

---



In [ ]:
doc_title_pairs = parquetFile.limit(1000).select("id", "title").rdd
doc_body_pairs = parquetFile.limit(1000).select("id", "text").rdd
doc_anchor_pairs = parquetFile.limit(1000).select("id", "anchor_text").rdd

# Helper functions

---

---





### *Tokenization*

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    return list_of_tokens

### *word_count*

In [ ]:
def word_count(id, text):
    ''' Count the frequency of each word in `text` (tf) that is not included in 
    `all_stopwords` and return entries that will go into our posting lists. 
    Parameters:
    -----------
    id: int
        Document id
    text: str
        Text of one document
    Returns:
    --------
    List of tuples
        A list of (token, (doc_id, tf)) pairs 
        for example: [("Anarchism", (12, 5)), ...]
    '''
    tokenized = tokenize(text)
    word_tf = Counter(tokenized)
    result = []
    for word, tf in word_tf.items():
        result.append((word, (id, tf)))
    return result

### *doc_len*

In [ ]:
def doc_len(id, text):
    ''' Calculate the length of the text
    Parameters:
    -----------
    id: int
        Document id
    text: str
        Text of one document
    Returns:
    --------
    A dict of (doc_id, doc_len) pairs 
    for example: {12:5, ...}
    '''
    tokenized = tokenize(text)
    result = []
    return [(id, len(tokenized))]

### *reduce_word_counts*

In [ ]:
def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
    unsorted_pl: list of tuples
        A list of (wiki_id, tf) tuples 
    Returns:
    --------
    list of tuples
        A sorted posting list.
    '''
    return sorted(unsorted_pl, key=lambda item: item[0])

### *calculate_df*

In [ ]:
def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
    postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
        An RDD where each element is a (token, df) pair.
    '''
    return postings.map(lambda x: (x[0],len(x[1])))

### *calculate_term_total*

In [ ]:
def calculate_term_total(postings):
    ''' Takes a posting list RDD and calculate the term total for each token.
    Parameters:
    -----------
    postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
        An RDD where each element is a (token, term_total) pair.
    '''

    return postings.mapValues(lambda posting: builtins.sum([tf for doc_id, tf in posting]))

### *Partitioning and writing the index*

In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, file_name):
    ''' A function that partitions the posting lists into buckets, writes out 
    all posting lists in a bucket to disk, and returns the posting locations for 
    each bucket. Partitioning should be done through the use of `token2bucket` 
    above. Writing to disk should use the function  `write_a_posting_list`, a 
    static method implemented in inverted_index_colab.py under the InvertedIndex 
    class. 
    Parameters:
    -----------
    postings: RDD
        An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
    RDD
        An RDD where each item is a posting locations dictionary for a bucket. The
        posting locations maintain a list for each word of file locations and 
        offsets its posting list was written to. See `write_a_posting_list` for 
        more details.
    '''
    buckets = postings.map(lambda item: (token2bucket_id(item[0]), item)).groupByKey()
    res = buckets.map(lambda item: InvertedIndex.write_a_posting_list(item, file_name))
    return res

# Create Indices

---



---



## Create **Title** Index

---



In [ ]:
################## Initialize Index ####################
title_index = InvertedIndex()

################## Posting Locs ########################
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1])) # -> [(token, (doc_id, tf)), ]
postings = word_counts.groupByKey().mapValues(reduce_word_counts)       # -> [(token, [(doc_id1, tf), (doc_id2, tf), ]), ]

posting_locs_list = partition_postings_and_write(postings, 'title_bins').collect()

super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)
# --------------------------------------------
title_index.posting_locs = super_posting_locs

################## DF ##################################
w2df = calculate_df(postings)

w2df_dict = w2df.collectAsMap()                                         # -> [(token, df), ..]
# --------------------------------------------
title_index.df = w2df_dict

################## Term Total ##########################
w2tf = calculate_term_total(postings)

w2tf_dict = dict(w2tf.collect())                                         # -> [(token, tf), ..]
# --------------------------------------------
title_index.term_total = w2tf_dict

################### Write ##############################
title_index.write_index('.', 'title_index')

## Create **Body** Index

---



In [ ]:
################## Initialize Index ####################
body_index = InvertedIndex()

################## Posting Locs ########################
word_counts = doc_body_pairs.flatMap(lambda x: word_count(x[0], x[1]))  # -> [(token, (doc_id, tf)), ]
postings = word_counts.groupByKey().mapValues(reduce_word_counts)       # -> [(token, [(doc_id1, tf), (doc_id2, tf), ]), ]

postings_filtered = postings.filter(lambda x: len(x[1]) > 50)           # -> [(token, [(doc_id1, tf), (doc_id2, tf), .. , (doc_id10, tf)]), ]

posting_locs_list = partition_postings_and_write(postings_filtered, 'body_bins').collect()

super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)
# --------------------------------------------
body_index.posting_locs = super_posting_locs

################## DF ##################################
w2df = calculate_df(postings_filtered)

w2df_dict = w2df.collectAsMap()                                         # -> [(token, df), ..]
# --------------------------------------------
body_index.df = w2df_dict

################## Term Total ##########################
w2tf = calculate_term_total(postings_filtered)

w2tf_dict = dict(w2tf.collect())                                        # -> [(token, tf), ..]
# --------------------------------------------
body_index.term_total = w2tf_dict

################## DL ##################################
doc2len = doc_body_pairs.flatMap(lambda x: doc_len(x[0], x[1]))         # -> [(doc1, 12), (doc2, 34)]
body_index.DL = doc2len.collectAsMap()

################### Write ##############################
body_index.write_index('.', 'body_index')

## Create **Anchor Text** Index

---

In [ ]:
################## Initialize Index ####################
anchor_index = InvertedIndex()

################## Posting Locs ########################
id2anchor = doc_anchor_pairs.flatMap(lambda x: x[1])
id2text = id2anchor.distinct().groupByKey().mapValues(lambda x: " ".join(x))

word_counts = id2text.flatMap(lambda x: word_count(x[0], x[1]))          # -> [(token, (doc_id, tf)), ]
postings = word_counts.groupByKey().mapValues(reduce_word_counts)        # -> [(token, [(doc_id1, tf), (doc_id2, tf), ]), ]

posting_locs_list = partition_postings_and_write(postings, 'anchor_bins').collect()

super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)
# --------------------------------------------
anchor_index.posting_locs = super_posting_locs

################## DF ##################################
w2df = calculate_df(postings)

w2df_dict = w2df.collectAsMap()                                          # -> [(token, df), ..]
# --------------------------------------------
anchor_index.df = w2df_dict

################## Term Total ##########################
w2tf = calculate_term_total(postings)

w2tf_dict = dict(w2tf.collect())                                         # -> [(token, tf), ..]
# --------------------------------------------
anchor_index.term_total = w2tf_dict

################### Write ##############################
anchor_index.write_index('.', 'anchor_index')

# Read Indices

---

---





In [ ]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
    with closing(MultiFileReader()) as reader:
        locs = inverted.posting_locs[w]
        b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
        posting_list = []
        for i in range(inverted.df[w]):
            doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
            tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
            posting_list.append((doc_id, tf))
        return posting_list

In [ ]:
def get_posting_gen(index):
    """
    This function returning the generator working with posting list.
    
    Parameters:
    ----------
    index: inverted index    
    """
    words, pls = zip(*index.posting_lists_iter())
    return words, pls